In [2]:
#####Ke Feng 
#####DSC478 Programming Machine Learning 
#####Assginment 2 Question 1

In [3]:
#####Import Libraries and Pre-Processing Before Building Classifiers 
#import libraries (Import all relevant libraries)
import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt

In [4]:
#load the data
#set up usecols as 1 for target class, no index column in the array
traindf = np.genfromtxt("trainMatrixModified.txt",delimiter="\t" )
testdf =np.genfromtxt("testMatrixModified.txt", delimiter = "\t")
traintar = np.genfromtxt("trainClasses.txt", delimiter = "\t", usecols=(1), dtype=(int))
testtar = np.genfromtxt("testClasses.txt", delimiter = "\t", usecols=(1), dtype=(int))
mterms=np.genfromtxt("modifiedterms.txt", delimiter="\t")

In [5]:
#Traspose the Data (Only Transpose the Training and Testing Sets, Leave the Class Sets Alone)
traindft=traindf.T
testdft=testdf.T

In [7]:
#Data Checking
print(traindft.shape)
print(testdft.shape)
print(traintar.shape)
print(testtar.shape)

(800, 5500)
(200, 5500)
(800,)
(200,)


In [8]:
#1a. Build a KNN classifier using Euclidian & Cosine distances. Return neighbors, distance, and predicted class
#reference: DePaul University Professor Mobasher's KNN tutoril http://facweb.cs.depaul.edu/mobasher/classes/CSC478/Notes/Docs-Terms-Distances.html
#x=instace, d=train_data, trc=train_label, k=kvalue, measure= E or C
def KNN(x, d, trc, k, measure):
    if measure == 0:
        distance = np.sqrt(((d - x)**2).sum(axis=1))  #Euclidian 
    elif measure == 1:
        D_norm = np.array([np.linalg.norm(d[i]) for i in range(len(d))])   #Consine 
        x_norm = np.linalg.norm(x)
        sims = np.dot(d, x)/(D_norm * x_norm)
        distance = 1 - sims
    idx = np.argsort(distance)  #Referencing Professor Mombasher's KNN tutorial 

#Define Predicted Class 
    classlabel = trc[idx[:k]]
    c0 = 0
    c1 = 0
    for i in classlabel:
        if trc[i] == 0:
            c0 += 1
        else:
            c1 += 1
        if c0 > c1:
            predclass= 0
        else:
            predclass = 1
            
#return neighbors, distance, and predicted class
    return idx[:k], sorted(distance)[:k], predclass

In [9]:
#Call Function and Print the Result 
idx, distance, predclass=KNN(testdft[1], traindft, traintar, 5, 1)
idx1, distance1, predclass1=KNN(testdft[1], traindft, traintar, 5, 0)

#print the Eculdian result 
print ("Result from measuring Euclidian Distance (ED)")
print ("\nThe Index of 5 Nearest Neighbors Are (ED): %s" % idx)
print ("\nThe Distances are (ED): %s" % distance)
print ("\nThe Predicted Class is(ED): %s" % predclass)
#print the Cosine result  
print ("Result from measuring Cosine Similarity (CS)")
print ("\nThe Index of 5 Nearest Neighbors Are (CS): %s" % idx1)
print ("\nThe Distances are (CS): %s" % distance1)
print ("\nThe Predicted Class is(CS): %s" % predclass1)

Result from measuring Euclidian Distance (ED)

The Index of 5 Nearest Neighbors Are (ED): [382 775 163 587 550]

The Distances are (ED): [0.6627393933148433, 0.670530750146896, 0.6780577758631312, 0.6938607233351928, 0.6975061044432607]

The Predicted Class is(ED): 0
Result from measuring Cosine Similarity (CS)

The Index of 5 Nearest Neighbors Are (CS): [798 554 757 224  38]

The Distances are (CS): [9.539392014169456, 9.643650760992955, 9.643650760992955, 9.695359714832659, 9.797958971132712]

The Predicted Class is(CS): 0


In [10]:
#Create a Function to compute the classification accuracy. 
#x=instance, d=train_data, trc=train_label, tel=test_label, k=kvalue, measure=E or C
def accuracy(x, d, trc, tel, k, measure):
    testlength=len(x)
    total = 0
    right = 0
    for i in range(testlength):
        total += 1
        idx, distance, predclass = KNN(x[i,:], d, trc, k, measure)
        if predclass == tel[i]:
            right += 1
    acc = float(right/total)
    return acc

In [11]:
acce = accuracy(testdft, traindft, traintar, testtar, 5, 0)
print ("The accuracy KNN (K=5) using Eclidian is %s" % acce)
accc=accuracy(testdft, traindft, traintar, testtar, 5, 1)
print ("The accuracy KNN (K=5) using Cosine is %s" % accc)

The accuracy KNN (K=5) using Eclidian is 0.815
The accuracy KNN (K=5) using Cosine is 0.97


In [12]:
#1c.Define K from 1-20 and find out the result
acc = pd.DataFrame()
for i in range(20):
    i+=1
    acc.loc[i,0] = i
    acc.loc[i,1] =accuracy(testdft, traindft, traintar, testtar, i, 0) #Euclidian
    acc.loc[i,2] =accuracy(testdft, traindft, traintar, testtar, i, 1) #Cosine
acc.columns = ['k', 'acce','accc']
acc

,k,acce,accc
1,1.0,0.780,0.985
2,2.0,0.675,0.980
3,3.0,0.810,0.970
4,4.0,0.770,0.980
5,5.0,0.815,0.970
6,6.0,0.835,0.975
7,7.0,0.765,0.980
8,8.0,0.800,0.975
9,9.0,0.750,0.975
10,10.0,0.860,0.985


In [13]:
#1d. Add term weights and test the result
#Reference Professor Mobasher's Term-Doc Matrix and TF*IDF: https://nbviewer.jupyter.org/url/bmobasher.com/Class/CSC478/Docs-Terms.ipynb
td = np.concatenate((traindf, testdf), axis = 1)
df = np.array([(td!=0).sum(1)]).T
num_docs = len(td[0,:])
nmatrix = np.ones(np.shape(td), dtype=float)*num_docs
idf = np.log2(np.divide(nmatrix, df))
#Count the term weight
tfidf = td*idf
ndata = tfidf.T

In [14]:
train2 = ndata[0:800,]
test2 = ndata[800:]

In [15]:
#Use the accuracy function above and compare the results
acc2 = pd.DataFrame()
for i in range(20):
    i+=1
    acc2.loc[i,0] = i
    acc2.loc[i,1] =accuracy(test2, train2, traintar, testtar, i, 0) #Euclidian
    acc2.loc[i,2] =accuracy(test2, train2, traintar, testtar, i, 1) #Cosine
acc2.columns = ['k2', 'acce2','accc2']
acc2

,k2,acce2,accc2
1,1.0,0.720,0.950
2,2.0,0.605,0.945
3,3.0,0.740,0.975
4,4.0,0.620,0.960
5,5.0,0.805,0.970
6,6.0,0.835,0.970
7,7.0,0.730,0.985
8,8.0,0.740,0.985
9,9.0,0.690,0.990
10,10.0,0.745,0.975
